<a href="https://colab.research.google.com/github/thecoder-001/DevColab/blob/master/DevColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```python
  ___           ___     _      _    
 |   \ _____ __/ __|___| |__ _| |__ 
 | |) / -_) V / (__/ _ \ / _` | '_ \
 |___/\___|\_/ \___\___/_\__,_|_.__/
                                   
```
**Run a Development environment on Google Colab!**

---
The script below will set up your dev environment and create a SSh tunnel. You'll have to get auth token from google drive and ngork to be able to use and store your files - don't worry, the scripts below will do the majority of the work for you.


In [ ]:
!pip install pyngrok

import os
from pyngrok import ngrok
# Update the package lists
!sudo apt update

# Don't want to update all the packages? Comment the below command. Warning: Updating usually takes some time (~1 minute)
!sudo apt upgrade

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Drive is mounted under "/content/drive/My Drive/"

# Ask for the ngrok authtoken
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
!ngrok authtoken $authtoken # login to ngrok
print("Google drive mounted and ngrok primed up. Now run any of the cells below.")


#### For VScode, run this:

In [ ]:
!curl -fsSL https://code-server.dev/install.sh | sh
vscode_tunnel = ngrok.connect(8123)
print("Open the first link below in browser:")
print(str(vscode_tunnel).replace('http://', 'https://'))

!mkdir "/content/drive/My Drive/code-server"
! sudo rm "/content/drive/My Drive/code-server/config.yaml"
! echo "bind-addr: 127.0.0.1:8123" >> "/content/drive/My Drive/code-server/config.yaml"
! echo "auth: password" >> "/content/drive/My Drive/code-server/config.yaml"
! echo "password: devcolab" >> "/content/drive/My Drive/code-server/config.yaml" #the default password is devcolab
! echo "cert: false" >> "/content/drive/My Drive/code-server/config.yaml"
! code-server --config "/content/drive/My Drive/code-server/config.yaml" --user-data-dir "/content/drive/My Drive/code-server"


#### For direct ssh access, run this
 Currently slightly broken, fixing soon

In [ ]:
# ssh setup
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
!echo root:devcolab | chpasswd #the default ssh password is devcolab, change if you want
!mkdir -p /var/run/sshd
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
!echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
!sudo /usr/sbin/sshd -D  &

# Connect to ngrok
ssh_tunnel = ngrok.connect(22, "tcp")
address = ((str(ssh_tunnel).split('"')[1::2])[0]).replace('tcp://', '')
print('Your tunnel address is ' + address)
print('Use the following command to connect: ' + "ssh root@" + "tcp://" + (address.split(':'))[0] + " -p " + (address.split(':'))[1])
print("or if that doesn't work, use this: " + "ssh root@" + (address.split(':'))[0] + " -p " + (address.split(':'))[1])



#### Debug:

In [ ]:
!cat ~/.config/code-server/config.yaml

In [ ]:
!pwd